In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from tqdm import tqdm
import time
from sklearn import metrics
import ast

In [2]:
df = pd.read_csv('./scrapping/last_crawler.csv')

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [5]:
df.drop(columns=['url_live'], inplace=True)

# Unpack lineup

In [6]:
df.lineupList = df.lineupList.apply(ast.literal_eval)

In [7]:
df[['homeLineup', 'awayLineup']] = pd.DataFrame(df.lineupList.tolist(), index = df.index)

In [8]:
df[['home_player_1',
'home_player_2',
'home_player_3',
'home_player_4',
'home_player_5',
'home_player_6',
'home_player_7',
'home_player_8',
'home_player_9',
'home_player_10',
'home_player_11'
]] = pd.DataFrame(df.homeLineup.tolist(), index = df.index).drop(columns=11)

In [9]:
df[['away_player_1',
'away_player_2',
'away_player_3',
'away_player_4',
'away_player_5',
'away_player_6',
'away_player_7',
'away_player_8',
'away_player_9',
'away_player_10',
'away_player_11'
]] = pd.DataFrame(df.awayLineup.tolist(), index = df.index).drop(columns=11)

In [10]:
df[['home_player_1',
'home_player_2',
'home_player_3',
'home_player_4',
'home_player_5',
'home_player_6',
'home_player_7',
'home_player_8',
'home_player_9',
'home_player_10',
'home_player_11',
'away_player_1',
'away_player_2',
'away_player_3',
'away_player_4',
'away_player_5',
'away_player_6',
'away_player_7',
'away_player_8',
'away_player_9',
'away_player_10',
'away_player_11']].head()

,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11
0,"(person-4802, Jefferson)","(person-16989, Alessandro)","(person-18379, Antônio Carlos)","(person-9777, Fábio Ferreira)","(person-71156, Somália)","(person-8947, Leandro Guerreiro)","(person-34721, Túlio Souza)","(person-35429, Fahel)","(person-10846, Renato Cajá)","(person-5883, G. Herrera)","(person-10395, S. Abreu)","(person-9341, Felipe)","(person-78657, Alex Sandro)","(person-39554, Bruno Aguiar)","(person-66249, Maranhão)","(person-9435, Durval)","(person-9473, Madson)","(person-16604, Roberto Brum)","(person-65382, Wesley)","(person-32454, Marquinhos)","(person-102697, Neymar)","(person-75403, André)"
1,"(person-9277, Marcos)","(person-9164, Vitor)","(person-64406, Léo)","(person-22373, P. Armero)","(person-9114, Cleiton Xavier)","(person-1984, Lincoln)","(person-9293, Pierre)","(person-9416, Edinho)","(person-32286, Marcio Araujo)","(person-43983, Robert)","(person-60231, Ewerthon)","(person-18048, J. Viáfara)","(person-84748, Reniê)","(person-9319, Egídio)","(person-33010, Wallace)","(person-33025, Bida)","(person-33014, Neto Coruja)","(person-76064, Uelliton)","(person-70702, Rafael Granja)","(person-9480, Renato Ribeiro)","(person-28985, Júnior Pipoca)","(person-71829, Elkeson)"
2,"(person-32707, Aranha)","(person-8908, Coelho)","(person-22975, J. Campos)","(person-74179, Werley)","(person-9285, Leandro)","(person-78303, Fabiano)","(person-4791, Ricardinho)","(person-19665, Corrêa)","(person-29124, Zé Luís)","(person-4493, Diego Tardelli)","(person-33018, Muriqui)","(person-16938, Fernando Prass)","(person-71699, Dedé)","(person-32825, Ramon)","(person-6884, Thiago Martinelli)","(person-73121, Magno Cruz)","(person-17174, Nilton)","(person-33254, Souza)","(person-76808, Paulinho)","(person-82507, Philippe Coutinho)","(person-8956, Dodô)","(person-34705, Elton)"
3,"(person-9781, Lauro)","(person-68732, Arilton)","(person-9348, Kleber)","(person-99990, Ronaldo Alves)","(person-5989, Fabiano Eler)","(person-75171, Ronaldo Conceição)","(person-20638, Giuliano)","(person-39549, Taison)","(person-32392, Glaydson)","(person-15264, P. Guiñazú)","(person-12927, Kléber Pereira)","(person-8961, Fábio)","(person-74810, Diego Renan)","(person-73000, Fernandinho)","(person-8974, Thiago Heleno)","(person-78857, Gil)","(person-23358, Fabrício)","(person-32916, Pedro Ken)","(person-9261, Elicarlos)","(person-530, Fabinho)","(person-32327, Wellington Paulista)","(person-33088, Kléber)"
4,"(person-8906, Edson)","(person-128779, Ayrton)","(person-78862, Gilson)","(person-81652, Chiquinho Baiano)","(person-33528, Welton Felipe)","(person-32493, Agenor)","(person-9479, Elias)","(person-33515, Pituca)","(person-33052, Ramalho)","(person-78865, Juninho)","(person-78511, Marcão)","(person-32329, Victor)","(person-4603, Rodrigo)","(person-71117, Edílson)","(person-115227, Neuton)","(person-32744, Ozéia)","(person-9209, Adílson)","(person-32350, Maylson)","(person-101371, Mithyuê)","(person-34725, Willian Magrão)","(person-9393, Hugo)","(person-110278, Bérgson)"


In [11]:
df.home_player_1 = df.home_player_1.astype('str')
df.home_player_2 = df.home_player_2.astype('str')
df.home_player_3 = df.home_player_3.astype('str')
df.home_player_4 = df.home_player_4.astype('str')
df.home_player_5 = df.home_player_5.astype('str')
df.home_player_6 = df.home_player_6.astype('str')
df.home_player_7 = df.home_player_7.astype('str')
df.home_player_8 = df.home_player_8.astype('str')
df.home_player_9 = df.home_player_9.astype('str')
df.home_player_10 = df.home_player_10.astype('str')
df.home_player_11 = df.home_player_11.astype('str')
df.away_player_1 = df.away_player_1.astype('str')
df.away_player_2 = df.away_player_2.astype('str')
df.away_player_3 = df.away_player_3.astype('str')
df.away_player_4 = df.away_player_4.astype('str')
df.away_player_5 = df.away_player_5.astype('str')
df.away_player_6 = df.away_player_6.astype('str')
df.away_player_7 = df.away_player_7.astype('str')
df.away_player_8 = df.away_player_8.astype('str')
df.away_player_9 = df.away_player_9.astype('str')
df.away_player_10 = df.away_player_10.astype('str')
df.away_player_11 = df.away_player_11.astype('str')

In [12]:
df.sublineupList = df.sublineupList.apply(ast.literal_eval)

In [13]:
df[['subHomeLineup', 'subAwayLineup']] = pd.DataFrame(df.sublineupList.tolist(), index = df.index)

In [14]:
df[['sub_home_player_1',
'sub_home_player_2',
'sub_home_player_3',
'sub_home_player_4',
'sub_home_player_5',
'sub_home_player_6',
'sub_home_player_7',
'sub_home_player_8',
'sub_home_player_9',
'sub_home_player_10',
'sub_home_player_11',
'sub_home_player_12'
]] = pd.DataFrame(df.subHomeLineup.tolist(), index = df.index).drop(columns=12)

In [15]:
df[['sub_away_player_1',
'sub_away_player_2',
'sub_away_player_3',
'sub_away_player_4',
'sub_away_player_5',
'sub_away_player_6',
'sub_away_player_7',
'sub_away_player_8',
'sub_away_player_9',
'sub_away_player_10',
'sub_away_player_11',
'sub_away_player_12'
]] = pd.DataFrame(df.subAwayLineup.tolist(), index = df.index)

In [16]:
df[['sub_home_player_1',
'sub_home_player_2',
'sub_home_player_3',
'sub_home_player_4',
'sub_home_player_5',
'sub_home_player_6',
'sub_home_player_7',
'sub_home_player_8',
'sub_home_player_9',
'sub_home_player_10',
'sub_home_player_11',
'sub_home_player_12',
'sub_away_player_1',
'sub_away_player_2',
'sub_away_player_3',
'sub_away_player_4',
'sub_away_player_5',
'sub_away_player_6',
'sub_away_player_7',
'sub_away_player_8',
'sub_away_player_9',
'sub_away_player_10',
'sub_away_player_11',
'sub_away_player_12']].tail()

,sub_home_player_1,sub_home_player_2,sub_home_player_3,sub_home_player_4,sub_home_player_5,sub_home_player_6,sub_home_player_7,sub_home_player_8,sub_home_player_9,sub_home_player_10,sub_home_player_11,sub_home_player_12,sub_away_player_1,sub_away_player_2,sub_away_player_3,sub_away_player_4,sub_away_player_5,sub_away_player_6,sub_away_player_7,sub_away_player_8,sub_away_player_9,sub_away_player_10,sub_away_player_11,sub_away_player_12
3406,"(person-9152, Harlei)","(person-171763, Moisés)","(person-233910, Alex Alves)","(person-78669, Valmir Lucas)","(person-113530, João Paulo)","(person-112156, Esquerdinha)","(person-291015, Erik)","(person-289857, Danilo)","(person-73008, Bruno Mineiro)",None,None,None,"(person-9434, Saulo)","(person-228302, Igor)","(person-282265, Danilo)","(person-211001, Oswaldo)","(person-279127, Willian)","(person-80981, Aílton)","(person-313877, Ronaldo)","(person-32296, Renan Oliveira)","(person-68267, Ananias)",None,None,None
3407,"(person-335343, Paulinho)","(person-395632, Everton)","(person-113650, Rafael Forster)","(person-354381, Murilo)","(person-401925, Deivid)","(person-7863, Ygor)","(person-9171, Felipe Menezes)","(person-236401, Liniker)","(person-171754, Wesley)","(person-153214, Ruan)","(person-396117, Carlos Eduardo)",None,"(person-32707, Aranha)","(person-71262, Jaílson)","(person-170536, Jackson)","(person-365417, Nathan)","(person-8912, Leandro Almeida)","(person-9106, Lucas)","(person-72026, João Paulo)","(person-180856, A. Allione)","(person-171544, Andrei Girotto)","(person-64341, Alecsandro)","(person-4987, Rafael Marques)","(person-14317, J. Cristaldo)"
3408,"(person-98909, João Carlos)","(person-430449, Guilherme)","(person-288072, Jeferson)","(person-63158, Fernandinho)","(person-302497, Luan Peres)","(person-410124, Ravanelli)","(person-226420, Fábio Braga)","(person-187565, Jádson)","(person-163621, Negueba)","(person-419013, Claudinho)","(person-66336, Lins)","(person-476222, Yuri)","(person-61115, Artur Moraes)","(person-74810, Diego Renan)","(person-121698, Douglas Grolli)","(person-120553, Neném)","(person-258914, Osman)","(person-405660, Lucas Marques)","(person-433625, Perotti)","(person-4968, Túlio de Melo)","(person-302812, Niltinho)",None,None,None
3409,"(person-32931, Fernando Leal)","(person-103416, Paulão)","(person-433714, Christian Savio)","(person-33019, Marquinhos)","(person-425526, Matheusinho)","(person-32954, Gerson Magrão)","(person-9139, David)","(person-171754, Wesley)","(person-491352, Ademir Santos)",None,None,None,"(person-78768, Rodolfo)","(person-428038, Marlon)","(person-335723, Paulo Ricardo)","(person-433032, Danielzinho)","(person-434994, Mateus Norton)","(person-390829, Dodi)","(person-324444, Everaldo)","(person-239932, Marcos Júnior)","(person-66297, Júnior Dutra)",None,None,None
3410,"(person-68943, Bruno Grassi)","(person-65193, Paulo Miranda)","(person-292652, Léo Gomes)","(person-10265, Marcelo Oliveira)","(person-9138, Cícero)","(person-282447, Thaciano)","(person-417745, Kaio)","(person-32552, Douglas)","(person-276507, Alisson)","(person-33226, Marinho)","(person-398248, Pepê)","(person-33287, Jael)","(person-531145, Vitor Eudes)","(person-97650, Rafael)","(person-80390, Manoel)","(person-282477, Marcelo Hermes)","(person-408424, Patrick Brey)","(person-478707, Ederson)","(person-69072, F. Mancuello)","(person-65376, Robinho)","(person-233956, Lucas Silva)","(person-48535, Thiago Neves)","(person-396310, David)",None


In [17]:
df.sub_home_player_1 =  df.sub_home_player_1.astype('str')
df.sub_home_player_2 =  df.sub_home_player_2.astype('str')
df.sub_home_player_3 =  df.sub_home_player_3.astype('str')
df.sub_home_player_4 =  df.sub_home_player_4.astype('str')
df.sub_home_player_5 =  df.sub_home_player_5.astype('str')
df.sub_home_player_6 =  df.sub_home_player_6.astype('str')
df.sub_home_player_7 =  df.sub_home_player_7.astype('str')
df.sub_home_player_8 =  df.sub_home_player_8.astype('str')
df.sub_home_player_9 =  df.sub_home_player_9.astype('str')
df.sub_home_player_10 = df.sub_home_player_10.astype('str')
df.sub_home_player_11 = df.sub_home_player_11.astype('str')
df.sub_home_player_12 = df.sub_home_player_12.astype('str')
df.sub_away_player_1 =  df.sub_away_player_1.astype('str')
df.sub_away_player_2 =  df.sub_away_player_2.astype('str')
df.sub_away_player_3 =  df.sub_away_player_3.astype('str')
df.sub_away_player_4 =  df.sub_away_player_4.astype('str')
df.sub_away_player_5 =  df.sub_away_player_5.astype('str')
df.sub_away_player_6 =  df.sub_away_player_6.astype('str')
df.sub_away_player_7 =  df.sub_away_player_7.astype('str')
df.sub_away_player_8 =  df.sub_away_player_8.astype('str')
df.sub_away_player_9 =  df.sub_away_player_9.astype('str')
df.sub_away_player_10 = df.sub_away_player_10.astype('str')
df.sub_away_player_11 = df.sub_away_player_11.astype('str')
df.sub_away_player_12 = df.sub_away_player_12.astype('str')

In [18]:
df.drop(columns=['lineupList',
                 'homeLineup',
                 'awayLineup',
                 'subHomeLineup',
                 'subAwayLineup',
                 'sublineupList'], inplace=True)

In [19]:
df.drop(columns=['eventList1st', 'eventList2nd'], inplace=True)

# Get results

In [20]:
df['result'] = np.nan

In [21]:
def getResult(row):
    if row.home_team_score > row.away_team_score:
        result = 'win'
    elif row.home_team_score == row.away_team_score:
        result = 'draw'
    else:
        result = 'lose'
    return result

In [22]:
%%time 
df.result = df.apply(getResult, axis=1)

CPU times: user 288 ms, sys: 16 ms, total: 304 ms
Wall time: 306 ms


In [23]:
df[['home_team_score', 'away_team_score', 'result']].head()

,home_team_score,away_team_score,result
0,3,3,draw
1,1,0,win
2,2,1,win
3,1,2,lose
4,0,0,draw


# Dashes to zeros

In [24]:
cols =  ['at_current_draws_away', 
  'at_current_draws_global', 
  'at_current_draws_home', 
  'at_current_loss_away', 
  'at_current_loss_global', 
  'at_current_loss_home', 
  'at_current_wins_away', 
  'at_current_wins_global', 
  'at_current_wins_home', 
  'at_games_without_draw_away', 
  'at_games_without_draw_global', 
  'at_games_without_draw_home', 
  'at_games_without_lose_away', 
  'at_games_without_lose_global', 
  'at_games_without_lose_home', 
  'at_games_without_win_away', 
  'at_games_without_win_global', 
  'at_games_without_win_home', 
  'ht_current_draws_away', 
  'ht_current_draws_global', 
  'ht_current_draws_home', 
  'ht_current_loss_away', 
  'ht_current_loss_global', 
  'ht_current_loss_home', 
  'ht_current_wins_away', 
  'ht_current_wins_global', 
  'ht_current_wins_home', 
  'ht_games_without_draw_away', 
  'ht_games_without_draw_global', 
  'ht_games_without_draw_home', 
  'ht_games_without_lose_away', 
  'ht_games_without_lose_global', 
  'ht_games_without_lose_home', 
  'ht_games_without_win_away', 
  'ht_games_without_win_global', 
  'ht_games_without_win_home' ]

In [25]:
df[cols] = df[cols].apply(lambda x: x.replace('-',0))

In [26]:
df[cols] = df[cols].astype('int')

# Possesion to int

In [27]:
df.drop(columns='a_Posse de bola', inplace=True)

In [28]:
df.rename(columns={'h_Posse de bola':'possession'}, inplace=True)

In [29]:
def dropPercent(percent):
    try:
        x = percent.strip('%')
    except:
        x = percent
    return x

In [30]:
df.possession = df.possession.apply(dropPercent)

In [31]:
df.possession = df.possession.astype('float')

# DATETIME TO INT

In [32]:
 meses={'abril':'04',
        'maio':'05',
        'junho':'06',
        'julho':'07',
        'agosto':'08',
        'setembro':'09',
        'outubro':'10',
        'novembro':'11',
        'dezembro':'12'}

In [33]:
def monthToNumber(date):
    for month, number in meses.items():
        date = date.replace(month, number)
    return date

In [34]:
df.datetime = df.datetime.apply(monthToNumber)

In [35]:
df.datetime = pd.to_datetime(df.datetime, format='%d %m %Y - %H:%M')

In [36]:
df.sort_values(by=['datetime'], inplace=True)

In [37]:
def to_integer(dt_time):
    return dt_time.dt.day + 100*dt_time.dt.month + 10000*dt_time.dt.year

In [38]:
df['int_date'] = 0

In [39]:
%%time
df.datetime = to_integer(df.datetime)

CPU times: user 10.5 ms, sys: 0 ns, total: 10.5 ms
Wall time: 10 ms


In [40]:
df.to_csv('soccerDB4.csv')